This code is designed to utilize the Sportsipy API to consolidate nfl player stats from the 2020 season.  The output is a csv file with player stats organized in a weekly basis.

In [86]:
# Import dependencies
import pandas as pd

In [87]:
# Import classes
from sportsipy.nfl.teams import Teams
from sportsipy.nfl.schedule import Schedule
from sportsipy.nfl.boxscore import Boxscore

In [88]:
# Iterate through nfl teams in the 2020 season and output each team abbreviation
team_abv = []
for team in Teams(year = 2020).dataframes.index:
      team_abv.append(team)

In [89]:
team_abv

['GNB',
 'BUF',
 'TAM',
 'OTI',
 'NOR',
 'KAN',
 'RAV',
 'SEA',
 'CLT',
 'RAI',
 'MIN',
 'PIT',
 'CRD',
 'CLE',
 'MIA',
 'ATL',
 'DAL',
 'HTX',
 'SDG',
 'DET',
 'SFO',
 'RAM',
 'CHI',
 'CAR',
 'WAS',
 'PHI',
 'NWE',
 'DEN',
 'CIN',
 'JAX',
 'NYG',
 'NYJ']

In [90]:
# Iterate through the nfl 2020 schedule using team_abv as the range to output a list of a list containing every game
team_schedule = []
for i in range(len(team_abv)):
    schedule = list(Schedule(team_abv[i], year = '2020'))
    team_schedule.append(schedule)

In [91]:
team_schedule

[[September 13 - MIN,
  September 20 - DET,
  September 27 - NOR,
  October 5 - ATL,
  October 18 - TAM,
  October 25 - HTX,
  November 1 - MIN,
  November 5 - SFO,
  November 15 - JAX,
  November 22 - CLT,
  November 29 - CHI,
  December 6 - PHI,
  December 13 - DET,
  December 19 - CAR,
  December 27 - OTI,
  January 3 - CHI,
  January 16 - RAM,
  January 24 - TAM],
 [September 13 - NYJ,
  September 20 - MIA,
  September 27 - RAM,
  October 4 - RAI,
  October 13 - OTI,
  October 19 - KAN,
  October 25 - NYJ,
  November 1 - NWE,
  November 8 - SEA,
  November 15 - CRD,
  November 29 - SDG,
  December 7 - SFO,
  December 13 - PIT,
  December 19 - DEN,
  December 28 - NWE,
  January 3 - MIA,
  January 9 - CLT,
  January 16 - RAV,
  January 24 - KAN],
 [September 13 - NOR,
  September 20 - CAR,
  September 27 - DEN,
  October 4 - SDG,
  October 8 - CHI,
  October 18 - GNB,
  October 25 - RAI,
  November 2 - NYG,
  November 8 - NOR,
  November 15 - CAR,
  November 23 - RAM,
  November 29 

In [92]:
# Limit schedule list to the regular season (first 16 games)
regular_season = []
for s in team_schedule:
    regular_season.append(s[0:16])

In [93]:
regular_season

[[September 13 - MIN,
  September 20 - DET,
  September 27 - NOR,
  October 5 - ATL,
  October 18 - TAM,
  October 25 - HTX,
  November 1 - MIN,
  November 5 - SFO,
  November 15 - JAX,
  November 22 - CLT,
  November 29 - CHI,
  December 6 - PHI,
  December 13 - DET,
  December 19 - CAR,
  December 27 - OTI,
  January 3 - CHI],
 [September 13 - NYJ,
  September 20 - MIA,
  September 27 - RAM,
  October 4 - RAI,
  October 13 - OTI,
  October 19 - KAN,
  October 25 - NYJ,
  November 1 - NWE,
  November 8 - SEA,
  November 15 - CRD,
  November 29 - SDG,
  December 7 - SFO,
  December 13 - PIT,
  December 19 - DEN,
  December 28 - NWE,
  January 3 - MIA],
 [September 13 - NOR,
  September 20 - CAR,
  September 27 - DEN,
  October 4 - SDG,
  October 8 - CHI,
  October 18 - GNB,
  October 25 - RAI,
  November 2 - NYG,
  November 8 - NOR,
  November 15 - CAR,
  November 23 - RAM,
  November 29 - KAN,
  December 13 - MIN,
  December 20 - ATL,
  December 26 - DET,
  January 3 - ATL],
 [Septemb

In [94]:
# Iterate through each team schedule in the regular season and output each boxscore for every team
boxscore_index_list = []
for team_sch in regular_season:
#     print(team_sch)
    for game in team_sch:
#         print(game.boxscore_index)  # Prints the boxscore URI for each game in a team's schedule
        boxscore = game.boxscore_index  # Returns an instance of the Boxscore class for this specific game
        boxscore_index_list.append(boxscore)

In [95]:
# 32 teams x 16 games = 512 boxscore URI
len(boxscore_index_list)

512

In [96]:
# Iterate through every boxscore URI to output each boxscore to a list
game_data_list=[]
for i in range(len(boxscore_index_list)):
    game_data = Boxscore(boxscore_index_list[i])
#     print(game_data)
    game_data_list.append(game_data)

In [97]:
len(game_data_list)

512

In [98]:
# Function that utilizes the Boxscore class properties ‘home_players’ and‘away_players’ 
# to concatenate eachindividual BoxscorePlayer dataframe
def home_away_split(game_data):
#     Home
    home_df = game_data.home_players[0].dataframe
    for player in game_data.home_players[1:]:
        home_df = pd.concat([home_df, player.dataframe], axis = 0)
    home_df['name'] = [x.name for x in game_data.home_players]
    home_df.set_index('name', inplace = True)
    
#     Away
    away_df = game_data.away_players[0].dataframe
    for player in game_data.away_players[1:]:
        away_df = pd.concat([away_df, player.dataframe], axis = 0)
    away_df['name'] = [x.name for x in game_data.away_players]
    away_df.set_index('name', inplace = True)

    return home_df, away_df

In [99]:
# Append boxscores for all games
full_df_home = []
full_df_away = []

# Adding a 'Week' column
x = 0
for i in range(len(game_data_list)):
    x = x+1
    if x <= 16:
        x = x
    else:
        x = 1
        
#     Data for home game x
    first_df, second_df = home_away_split(game_data_list[i])
    first_df['week'] = x
    second_df['week'] = x
    full_df_home.append(first_df)
    full_df_away.append(second_df)

In [100]:
# Concatnate all home boxscores
homebox_df = pd.concat(full_df_home)
homebox_df.tail(10)

,completed_passes,attempted_passes,passing_yards,passing_touchdowns,interceptions_thrown,times_sacked,yards_lost_from_sacks,longest_pass,quarterback_rating,rush_attempts,...,longest_punt_return,extra_points_made,extra_points_attempted,field_goals_made,field_goals_attempted,punts,total_punt_yards,yards_per_punt,longest_punt,week
name,,,,,,,,,,,,,,,,,,,,,
Akeem Spence,None,None,None,None,None,None,None,None,NaN,None,...,None,None,None,None,None,None,None,None,None,16
Rashod Berry,None,None,None,None,None,None,None,None,NaN,None,...,None,None,None,None,None,None,None,None,None,16
Michael Jackson,None,None,None,None,None,None,None,None,NaN,None,...,None,None,None,None,None,None,None,None,None,16
Adrian Phillips,None,None,None,None,None,None,None,None,NaN,None,...,None,None,None,None,None,None,None,None,None,16
John Simon,None,None,None,None,None,None,None,None,NaN,None,...,None,None,None,None,None,None,None,None,None,16
Michael Onwenu,None,None,None,None,None,None,None,None,NaN,None,...,None,None,None,None,None,None,None,None,None,16
Donte Moncrief,None,None,None,None,None,None,None,None,NaN,None,...,0,None,None,None,None,None,None,None,None,16
J.J. Taylor,None,None,None,None,None,None,None,None,NaN,None,...,0,None,None,None,None,None,None,None,None,16
Nick Folk,None,None,None,None,None,None,None,None,NaN,None,...,None,4,4,None,None,0,0,None,0,16


In [101]:
len(homebox_df)

15880

In [102]:
# # Append boxscores for all away games
# full_df_away = []

# # Adding a 'Week' column
# x = 0
# for i in range(len(game_data_list)):
#     x = x+1
#     if x <= 16:
#         x = x
#     else:
#         x = 1
        
# #     Data for home game x
#     first_df, second_df = home_away_split(game_data_list[i])
#     second_df['week'] = x
#     full_df_away.append(second_df)

In [103]:
# Concatnate all away boxscores
awaybox_df = pd.concat(full_df_away)
awaybox_df.tail(10)

,completed_passes,attempted_passes,passing_yards,passing_touchdowns,interceptions_thrown,times_sacked,yards_lost_from_sacks,longest_pass,quarterback_rating,rush_attempts,...,longest_punt_return,extra_points_made,extra_points_attempted,field_goals_made,field_goals_attempted,punts,total_punt_yards,yards_per_punt,longest_punt,week
name,,,,,,,,,,,,,,,,,,,,,
Corey Ballentine,None,None,None,None,None,None,None,None,NaN,None,...,0,None,None,None,None,None,None,None,None,16
Noah Dawkins,None,None,None,None,None,None,None,None,NaN,None,...,None,None,None,None,None,None,None,None,None,16
Folorunso Fatukasi,None,None,None,None,None,None,None,None,NaN,None,...,None,None,None,None,None,None,None,None,None,16
Sharif Finch,None,None,None,None,None,None,None,None,NaN,None,...,None,None,None,None,None,None,None,None,None,16
John Franklin-Myers,None,None,None,None,None,None,None,None,NaN,None,...,None,None,None,None,None,None,None,None,None,16
Ryan Griffin,None,None,None,None,None,None,None,None,NaN,None,...,None,None,None,None,None,None,None,None,None,16
J.T. Hassell,None,None,None,None,None,None,None,None,NaN,None,...,None,None,None,None,None,None,None,None,None,16
Tanzel Smart,None,None,None,None,None,None,None,None,NaN,None,...,None,None,None,None,None,None,None,None,None,16
Chase McLaughlin,None,None,None,None,None,None,None,None,NaN,None,...,None,2,2,None,None,0,0,None,0,16


In [104]:
len(awaybox_df)

15864

In [105]:
# Concatnate all game for 2020 season
frames = [homebox_df, awaybox_df]
season_df = pd.concat(frames)

In [106]:
len(season_df)

31744

In [117]:
season_df.head(10)

,completed_passes,attempted_passes,passing_yards,passing_touchdowns,interceptions_thrown,times_sacked,yards_lost_from_sacks,longest_pass,quarterback_rating,rush_attempts,...,longest_punt_return,extra_points_made,extra_points_attempted,field_goals_made,field_goals_attempted,punts,total_punt_yards,yards_per_punt,longest_punt,week
name,,,,,,,,,,,,,,,,,,,,,
Kirk Cousins,19,25,259,2,1,2,11,37,118.6,4,...,0,0,0,0,0,0,0,0.0,0,1
Alexander Mattison,0,0,0,0,0,0,0,0,0.0,6,...,0,0,0,0,0,0,0,0.0,0,1
Dalvin Cook,0,0,0,0,0,0,0,0,0.0,12,...,0,0,0,0,0,0,0,0.0,0,1
Adam Thielen,0,0,0,0,0,0,0,0,0.0,0,...,0,0,0,0,0,0,0,0.0,0,1
Olabisi Johnson,0,0,0,0,0,0,0,0,0.0,0,...,0,0,0,0,0,0,0,0.0,0,1
Kyle Rudolph,0,0,0,0,0,0,0,0,0.0,0,...,0,0,0,0,0,0,0,0.0,0,1
Justin Jefferson,0,0,0,0,0,0,0,0,0.0,0,...,0,0,0,0,0,0,0,0.0,0,1
Irv Smith Jr.,0,0,0,0,0,0,0,0,0.0,0,...,0,0,0,0,0,0,0,0.0,0,1
Tajae Sharpe,0,0,0,0,0,0,0,0,0.0,0,...,0,0,0,0,0,0,0,0.0,0,1


In [107]:
# Replace missing values with 0
season_df = season_df.fillna(0)

In [108]:
# Number of rows with at least one missing value
season_df.isnull().sum()

completed_passes                        0
attempted_passes                        0
passing_yards                           0
passing_touchdowns                      0
interceptions_thrown                    0
times_sacked                            0
yards_lost_from_sacks                   0
longest_pass                            0
quarterback_rating                      0
rush_attempts                           0
rush_yards                              0
rush_touchdowns                         0
longest_rush                            0
times_pass_target                       0
receptions                              0
receiving_yards                         0
receiving_touchdowns                    0
longest_reception                       0
fumbles                                 0
fumbles_lost                            0
interceptions                           0
yards_returned_from_interception        0
interceptions_returned_for_touchdown    0
longest_interception_return       

In [109]:
from sportsipy.mlb.roster import Roster

astros = Roster('HOU')
for player in astros.players:
    # Prints the name of all players who played for the Astros in the most
    # recent season.
    print(player.name)



Martín Maldonado
Yuli Gurriel
Jose Altuve
Carlos Correa
Alex Bregman
Michael Brantley
Myles Straw
Kyle Tucker
Yordan Álvarez
Chas McCormick
Aledmys Díaz
Jason Castro
Jake Meyers
Abraham Toro
Robel García
Taylor Jones
Jose Siri
Garrett Stubbs
Marwin Gonzalez
Jacob Wilson
Alex De Goti
Ronnie Dawson
Luis Garcia
Lance McCullers Jr.
José Urquidy
Framber Valdez
Zack Greinke
Jake Odorizzi
Blake Taylor
Kendall Graveman
Andre Scrubb
Brooks Raley
Tyler Ivey
Ryan Pressly
Josh James
Ralph Garza
Yimi García
Ryne Stanek
Joe Smith
Kent Emanuel
Seth Martinez
Peter Solomon
Cristian Javier
Austin Pruitt
Nivaldo Rodríguez
Phil Maton
Brandon Bielak
Rafael Montero
Pedro Báez
Enoli Paredes
Ryan Hartman
Bryan Abreu


In [5]:
from sportsipy.nfl.roster import Roster as nfl

astros = nfl('RAV')


In [6]:
for player in astros.players:
    # Prints the name of all players who played for the Astros in the most
    # recent season.
    print(player.position)

TypeError: 'NoneType' object is not subscriptable

In [115]:
playername_df = season_df.index.tolist()

In [120]:
set(playername_df)

{'Emmanuel Sanders',
 'Scott Miller',
 'Auden Tate',
 'Chris Moore',
 'Khalil Dorsey',
 'Jamie Collins',
 'Kenny Young',
 'Chuck Clark',
 'Jon Weeks',
 'Jake Elliott',
 'Hunter Niswander',
 'Jeremy Cox',
 'Jordan Akins',
 'Drew Sample',
 'Dion Dawkins',
 'DaQuan Jones',
 'Carter Coughlin',
 'Khaleke Hudson',
 'Ben Ellefson',
 'Tyson Alualu',
 'Jamil Douglas',
 'Hunter Bradley',
 'Daniel Carlson',
 'Zach Sieler',
 'Carl Granderson',
 'Andrew Brown',
 'John Wolford',
 'Jordan Miller',
 'Eric Wilson',
 'Matt Barkley',
 'Mario Addison',
 'Kyle Wilber',
 'Miles Sanders',
 'Davion Taylor',
 'Tevaughn Campbell',
 'Dan Chisena',
 'Marcus Maye',
 'Carson Wentz',
 'Kylie Fitts',
 "De'Michael Harris",
 'Brian Hill',
 'Tyler Conklin',
 'Danny Johnson',
 'Jimmy Moreland',
 'Robert Jackson',
 'Calais Campbell',
 'Michael Dickson',
 'Reid Ferguson',
 'Nick Vigil',
 'Ezekiel Elliott',
 'Montravius Adams',
 'Ty Johnson',
 'Yannick Ngakoue',
 'Linval Joseph',
 'Braxton Berrios',
 'Tony Jones',
 'Aaron L

In [113]:
from sportsipy.nfl.roster import Player

brees = Player('Drew Brees')
print(brees.name)  # Prints 'Drew Brees'
print(brees.passing_yards)  # Prints Brees' career passing yards
# Prints a Pandas DataFrame of all relevant stats per season for Brees
print(brees.dataframe)

None
None
None


In [10]:
from sportsipy.nfl.teams import Teams
DF2 = []
for team in Teams(2020):
    roster = nfl('RAV')
    for player in roster.players:
        print(player._position)
#         DF2.append(
#             {
#                 'id': player.player_id,
#                 'pos': player.position
#             })


['', '', '', 'te', '']
['', '', '', 'cb', '']
['', '']
['RB', '', 'RB', 'RB', 'rb', 'RB', None, 'RB', '', '', '']
['', '', '', '', '']
['', '', '', '', 'olb', '']
['WR', 'WR', '', '']
['', 'LG', 'LG', 'ol', '']
['wr', 'WR', 'wr', '']
['', '', 'rolb', '', 'RLB', 'LLB', '', 'MLB', 'il/LILB', 'LLB', 'lb', '']
['', 'LDE', 'LDE', 'LDE', 'LDE', 'LDE', 'LDE', 'LDE', 'LDE', 'RDE', 'RDE', 'DE/RDE', 'LDE', 'de', '']
['', '', 'FS', 'SS', 'saf', '']
['', '', '']
['', 'wr', '']
['', 'FS', 'saf', '']
['LDT', 'RDT', '', 'RDT', '', '', '', 'dt', '']
['LOLB', '', '', '']
['', 'RB', 'RB', 'RB', '', 'RB', '', 'rb', '']
['rilb', 'lb', '']
['LOLB', 'LOLB', 'LOLB', 'LOLB', 'LOLB', 'lolb', 'LOLB', 'ROLB', 'LDE', 'LDE', 'olb', '']
['', 'RCB', 'LCB', 'LCB', 'cb', '']
['', '', '']
['qb', 'QB', 'QB', 'qb', '']
['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'p']
['', '', '', '', '', '', '', '', '', '', '']
['', 'dt', '']
['', 'RDE', '', '', 'ROLB', '', '', '', 'lolb', 'LOLB', '', '']
None
['', '

['', '', '', 'te', '']
['', '', '', 'cb', '']
['', '']
['RB', '', 'RB', 'RB', 'rb', 'RB', None, 'RB', '', '', '']
['', '', '', '', '']
['', '', '', '', 'olb', '']
['WR', 'WR', '', '']
['', 'LG', 'LG', 'ol', '']
['wr', 'WR', 'wr', '']
['', '', 'rolb', '', 'RLB', 'LLB', '', 'MLB', 'il/LILB', 'LLB', 'lb', '']
['', 'LDE', 'LDE', 'LDE', 'LDE', 'LDE', 'LDE', 'LDE', 'LDE', 'RDE', 'RDE', 'DE/RDE', 'LDE', 'de', '']
['', '', 'FS', 'SS', 'saf', '']
['', '', '']
['', 'wr', '']
['', 'FS', 'saf', '']
['LDT', 'RDT', '', 'RDT', '', '', '', 'dt', '']
['LOLB', '', '', '']
['', 'RB', 'RB', 'RB', '', 'RB', '', 'rb', '']
['rilb', 'lb', '']
['LOLB', 'LOLB', 'LOLB', 'LOLB', 'LOLB', 'lolb', 'LOLB', 'ROLB', 'LDE', 'LDE', 'olb', '']
['', 'RCB', 'LCB', 'LCB', 'cb', '']
['', '', '']
['qb', 'QB', 'QB', 'qb', '']
['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'p']
['', '', '', '', '', '', '', '', '', '', '']
['', 'dt', '']
['', 'RDE', '', '', 'ROLB', '', '', '', 'lolb', 'LOLB', '', '']
None
['', '

KeyboardInterrupt: 